# **Libraries**

In [1]:
# import library
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import ADASYN
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# **Importation des données**  

In [ ]:
# Ajustez le chemin selon l'emplacement réel de votre fichier
data = pd.read_csv("guillaume.txt", sep=";")

# **Traitement de données**

In [3]:
def nettoyer_colonnes_par_indices(data, indices_colonnes):
    """
    Nettoie plusieurs colonnes spécifiées par leurs indices en supprimant les espaces,
    convertissant les valeurs en numérique, supprimant les valeurs non valides,
    et en les convertissant en entiers.

    Args:
        data (pd.DataFrame): Le DataFrame à traiter.
        indices_colonnes (list): Liste des indices des colonnes à nettoyer.

    Returns:
        pd.DataFrame: Le DataFrame avec les colonnes nettoyées.
    """
    # Parcourir les indices des colonnes
    for indice in indices_colonnes:
        # Obtenir le nom de la colonne depuis l'indice
        colonne = data.columns[indice]

        # Supprimer les espaces
        data[colonne] = data[colonne].astype(str).str.strip()
        # Convertir en numérique (remplace les erreurs par NaN)
        data[colonne] = pd.to_numeric(data[colonne], errors='coerce')
        # Supprimer les lignes avec des NaN dans la colonne
        data = data.dropna(subset=[colonne])
        # Convertir les valeurs en entiers
        data[colonne] = data[colonne].astype(int)

    return data

In [ ]:
# Liste des indices des colonnes à nettoyer
indices_colonnes = [1,2,5,6,7,8,9,15,16,22]

# Nettoyage des colonnes
data = nettoyer_colonnes_par_indices(data, indices_colonnes)

In [ ]:
# verifier les types de mes données
data.dtypes

In [ ]:
col_flot = ["Montant", "TauxImpNb_RB","TauxImpNB_CPM", "CA3TRetMtt", "CA3TR","ScoringFP1", 	"ScoringFP2", 	"ScoringFP3"]
col_str = ["ZIBZIN","IDAvisAutorisationCheque"]
co_date = ["DateTransaction"]
col_heure = ["Heure"]
# recuperer toutes les colonnnes qui ne foont pas parti de col_flot, col_str, col_heure, col_date dans col_int
col_int = [col for col in data.columns if col not in col_flot and col not in col_str and col not in col_heure and col not in co_date]
# afficher le nombre de chacune des liste
print("col_flot :", len(col_flot))
print("col_str :", len(col_str))
print("col_int :", len(col_int))
print("co_date :", len(co_date))
print("col_heure :", len(col_heure))

In [7]:
for col in col_flot:
    # Remplacer les virgules par des points dans les colonnes de type string
    data[col] = data[col].astype(str).str.replace(',', '.', regex=False)

    # Convertir les colonnes au type float
    data[col] = data[col].astype(float)

In [8]:
# Convertir en int
for col in col_int:
    # Vérifier si la colonne n'est pas déjà de type int
    if data[col].dtype != 'int64':
        # Remplacer les virgules par des points, si nécessaire
        data[col] = data[col].astype(str).str.replace(',', '.')
        # Convertir en type float puis en int (pour éviter les problèmes avec les décimales)
        data[col] = data[col].astype(float).astype(int)

In [9]:
# convertir la col_date en type date
data['DateTransaction'] = pd.to_datetime(data['DateTransaction'])


In [10]:
# Conversion des secondes depuis minuit en format timedelta
data['Heure'] = pd.to_timedelta(data['Heure'], unit='s')

# Convertir en format HH:MM:SS en chaîne (ou datetime.time si nécessaire)
data['Heure'] = data['Heure'].apply(lambda x: (pd.Timestamp("00:00:00") + x).time())

# **Analyse Exploratoire des Données**

In [ ]:
# liste des elements en flagImpaye
data['FlagImpaye'].value_counts()

# Graphique des transactions circulaires flagImpaye
plt.figure(figsize=(6, 6))
data['FlagImpaye'].value_counts().plot.pie(autopct='%1.1f%%')
plt.title('Répartition des transactions')
plt.show()

In [12]:
# selectionner toutes les colonnes sauf DateTransaction et Heure
NUMERIC_FEATURES = [col for col in data.columns if col not in ['DateTransaction', 'Heure','ZIBZIN','IDAvisAutorisationCheque']]

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Filtrer les transactions refusées
data_refuse = data[data['FlagImpaye'] == 1]

# Extraire le jour de la semaine et le mois
data_refuse['JourSemaine'] = data_refuse['DateTransaction'].dt.day_name()
data_refuse['Mois'] = data_refuse['DateTransaction'].dt.strftime('%b')  # Nom abrégé du mois

# Grouper par mois et jour de la semaine et compter les transactions refusées
grouped_data = data_refuse.groupby(['Mois', 'JourSemaine']).size().unstack()

# Ordonner les mois et les jours de la semaine
mois_ord = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
jours_ord = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Réorganiser les données selon l'ordre voulu
grouped_data = grouped_data.reindex(mois_ord, axis=0)
grouped_data = grouped_data[jours_ord]

# Création du graphique en barres groupées
plt.figure(figsize=(12, 6))
grouped_data.plot(kind='bar', figsize=(12, 6), rot=0)

# Ajouter des labels et un titre
plt.title('Transactions refusées par Mois et Jour de la Semaine')
plt.xlabel('Mois')
plt.ylabel('Nombre de Transactions Refusées')
plt.legend(title='Jour de la semaine')

# Afficher le graphique
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Création du boxplot avec une palette de couleurs améliorée
plt.figure(figsize=(12, 6))
sns.boxplot(x='FlagImpaye', y='Montant', data=data, palette=['#2E86C1', '#E74C3C'])  # Bleu pour 0, Rouge pour 1

# Ajout du titre et des labels
plt.title('Montant des Transactions selon le FlagImpaye', fontsize=14, fontweight='bold', pad=15)
plt.xlabel('FlagImpaye', fontsize=12)
plt.ylabel('Montant (€)', fontsize=12)

# Affichage du graphique
plt.show()


In [ ]:
# Ajoutons le jour de la semaine
data_refuse['JourSemaine'] = data_refuse['DateTransaction'].dt.day_name()

# Analyse croisée : jour de la semaine par mois
plt.figure(figsize=(15, 8))
cross_tab = pd.crosstab(data_refuse['Mois'], data_refuse['JourSemaine'])
sns.heatmap(cross_tab, cmap='YlOrRd', annot=True, fmt='d')
plt.title('Répartition des transactions refusées par jour de la semaine et par mois')
plt.xlabel('Jour de la semaine')
plt.ylabel('Mois')
plt.show()


In [ ]:
# distrubution des transactions selon flagImpaye, calcul le pourcentage de chaque valeur
data['FlagImpaye'].value_counts(normalize=True) * 100

# Nombre et pourcentage de transactions de chaque classe
nombre_transactions = data['FlagImpaye'].value_counts()
pourcentage_transactions = data['FlagImpaye'].value_counts(normalize=True) * 100
#afficher le nombre et le pourcentage dans un tableau
pd.DataFrame({'Nombre de transactions': nombre_transactions, 'Pourcentage de transactions (%)': pourcentage_transactions})






# **Partie 2**

### **Séparation des features et la cible**


In [17]:
# Définir les plages de dates pour l'apprentissage et le test
train_start_date = "2017-02-01"
train_end_date = "2017-08-31"
test_start_date = "2017-09-01"
test_end_date = "2017-11-30"

In [18]:
# Définir les plages de dates pour l'apprentissage et le test
train_start_date = "2017-02-01"
train_end_date = "2017-08-31"
test_start_date = "2017-09-01"
test_end_date = "2017-11-30"

# Séparer les ensembles d'apprentissage et de test
train_data = data[(data['DateTransaction'] >= train_start_date) & (data['DateTransaction'] <= train_end_date)]
test_data = data[(data['DateTransaction'] >= test_start_date) & (data['DateTransaction'] <= test_end_date)]

# Sélection des features et de la cible
X_train = train_data.drop(['IDAvisAutorisationCheque', 'ZIBZIN', 'Heure', 'FlagImpaye', 'DateTransaction'], axis=1)
y_train = train_data['FlagImpaye']
X_test = test_data.drop(['IDAvisAutorisationCheque', 'ZIBZIN', 'Heure', 'FlagImpaye', 'DateTransaction'], axis=1)
y_test = test_data['FlagImpaye']
amounts_test = test_data['Montant']

In [ ]:
# Vérifier les dimensions
print(f"Dimensions des données d'entraînement: {X_train.shape}, {y_train.shape}")
print(f"Dimensions des données de test: {X_test.shape}, {y_test.shape}")

In [20]:
# Définition de la matrice des coûts
def compute_margin(y_true, y_pred, amounts):
    r = 0.05  # Taux de marge
    margin = 0
    
    for yt, yp, m in zip(y_true, y_pred, amounts):
        if yt == 0 and yp == 0:  # TN : transaction acceptée et correcte
            margin += r * m
        elif yt == 1 and yp == 0:  # FN : transaction frauduleuse acceptée
            if m <= 20:
                loss = 0
            elif m <= 50:
                loss = 0.2 * m
            elif m <= 100:
                loss = 0.3 * m
            elif m <= 200:
                loss = 0.5 * m
            else:
                loss = 0.8 * m
            margin -= loss
        elif yt == 0 and yp == 1:  # FP : transaction correcte refusée
            margin += 0.7 * r * m
        # TP (fraude bien détectée) n'apporte ni perte ni gain
    
    return margin

In [ ]:
"""# Liste des modèles à tester
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'),
    "XGBoost": XGBClassifier(eval_metric='logloss', random_state=42, learning_rate=0.1, n_estimators=100),
    "Neural Network": MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
}

# Tester chaque modèle
results = {}
for name, model in models.items():
    print(f"\nEntraînement du modèle: {name}")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    margin = compute_margin(y_test, y_pred, amounts_test)
    results[name] = {
        "F1-score": f1,
        "Marge": margin,
        "Matrice de confusion": cm
    }
    print(f"F1-score: {f1:.4f}")
    print(f"Marge générée: {margin:.2f}")
    print("Matrice de confusion:")
    print(cm)"""


In [30]:
# Paramètres de la recherche sur grille
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 9],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}


In [31]:
# Création et configuration du modèle XGBoost de base
gxboost = XGBClassifier(eval_metric='logloss', random_state=42)

# Configuration de la recherche sur grille
grid_search = GridSearchCV(
    estimator=gxboost,
    param_grid=param_grid,
    scoring='f1',
    cv=3,
    verbose=2,
    n_jobs=-1
)


In [ ]:
# Entraînement de la recherche sur grille
print("Début de la recherche sur grille...")
grid_search.fit(X_train, y_train)

# Récupération du meilleur modèle
best_xgboost = grid_search.best_estimator_

# Affichage des meilleurs paramètres
print("\nMeilleurs paramètres trouvés:")
for param, value in grid_search.best_params_.items():
    print(f"{param}: {value}")

In [ ]:
# Évaluation du modèle
models = {
    "Optimized XGBoost": best_xgboost
}

# Fonction pour afficher la matrice de confusion
def plot_confusion_matrix(cm, title='Matrice de Confusion'):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(title)
    plt.ylabel('Vraie classe')
    plt.xlabel('Classe prédite')
    plt.show()

In [ ]:
# Test des modèles
results = {}
for name, model in models.items():
    print(f"\nEntraînement du modèle: {name}")
    
    # Prédictions et métriques
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    margin = compute_margin(y_test, y_pred, amounts_test)
    
    # Stockage des résultats
    results[name] = {
        "F1-score": f1,
        "Marge": margin,
        "Matrice de confusion": cm
    }
    
    # Affichage des résultats
    print(f"F1-score: {f1:.4f}")
    print(f"Marge générée: {margin:.2f}")
    print("Matrice de confusion:")
    print(cm)
    
    # Visualisation de la matrice de confusion
    plot_confusion_matrix(cm, f'Matrice de Confusion - {name}')



In [ ]:
# Affichage de l'importance des caractéristiques
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': best_xgboost.feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance)
plt.title('Importance des caractéristiques')
plt.show()